# News Analysis

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import operator
import math
import progressbar
import json as j
from unidecode import unidecode
import networkx as nx
from gb.notebook import *
import gb.synonyms.synonyms as syn
import gb.nlp.parser as par
from gb.metrics.hyper_similarity import HyperSimilarity

### HyperGraph, etc.

In [3]:
MAX_PROB = -12
hg = init_hypergraph('../reddit-worldnews-01012013-01082017-new-inference3.hg')
hs = HyperSimilarity(hg)
parser = par.Parser()

### NLP funs

In [4]:
def is_compound_propn(edge):
    if len(edge) < 3:
        return False
    if edge[0] != '+/gb':
        return False
    for item in edge[1:]:
        if is_symbol(item):
            if symbol_namespace(item)[-5:] != 'propn':
                return False
        elif not is_compound_propn(item):
            return False
    return True

def is_adj(edge):
    if is_edge(edge):
        return False
    return symbol_namespace(edge)[-3:] == 'adj'

def is_adp(edge):
    return is_symbol(edge) and symbol_namespace(edge)[-3:] == 'adp'

def is_noun(edge):
    if is_edge(edge):
        return False
    return symbol_namespace(edge)[-5:] == 'propn' or symbol_namespace(edge)[-4:] == 'noun'

def has_noun(edge):
    if is_edge(edge):
        for item in edge:
            if has_noun(item):
                return True
    else:
        return is_noun(edge)
    return False

def is_adjective_and_noun(edge):
    if len(edge) != 3:
        return False
    if edge[0] != '+/gb':
        return False
    if not is_adj(edge[1]):
        return False
    if not is_noun(edge[2]):
        return False
    return True

def is_cconj(edge):
    if is_edge(edge):
        return False
    return symbol_namespace(edge)[-5:] == 'cconj'

### Synonym funs

In [5]:
def syn_id(root):
    symbs = hg.symbols_with_root(root)
    for symb in symbs:
        if symbol_namespace(symb)[:3] == 'syn':
            return symb
    return None
        
def syn_label(syn_id):
    symbs = syn.synonyms(hg, syn_id)
    labels = []
    for symb in symbs:
        labels.append(hg.get_label(symb))
    best = None
    for label in labels:
        if best is None or len(label) < len(best):
            best = label
    parts = best.split('|')
    if len(parts) > 1:
        max_size = 0
        for part in parts:
            if len(part) > max_size:
                best = part.strip()
                max_size = len(best)
        
    return unidecode(best)

### Topic funs

In [7]:
def is_topic(symbol):
    if is_symbol(symbol) and symbol_namespace(symbol)[:3] == 'syn':
        root = symbol_root(symbol)
        return root[0].isalnum() and root[-1].isalnum()
    return False

def subtopics(edge):
    stopics = set()
    if is_edge(edge):
        for item in edge:
            topic = syn.main_synonym(hg, item)
            if is_topic(topic):
                stopics.add(item)
    return stopics

def choose_topic1(edge):
    topic = syn.main_synonym(hg, edge)
    if is_edge(edge):
        if is_compound_propn(edge) or is_adjective_and_noun(edge):
            return topic
        stopics = subtopics(edge)
        if len(stopics) == 1:
            return choose_topic1(next(iter(stopics)))    
    return topic

def choose_topic(edge):
    if is_edge(edge) and edge[0] == '+/gb' and is_adp(edge[1]):
        if len(edge) == 3:
            return choose_topic(edge[2])
        elif len(edge) > 3:
            return choose_topic1(edge)
    return syn.main_synonym(hg, edge)

def common_word(edge):
    syns = syn.synonyms(hg, edge)
    for s in syns:
        label = symbol2str(s)
        if ' ' in label:
            return False
        word = parser.make_word(label)
        if word.prob < MAX_PROB:
            return False
    return True

def atomic_topic(edge):
    atopic = syn.main_synonym(hg, edge)
    if is_topic(atopic) and not common_word(atopic):
        return {atopic}
    return set()

def atomic_topics(edge):
    atopics = atomic_topic(edge)
    if is_edge(edge):
        if len(edge) == 3 and edge[0] == '+/gb' and is_adp(edge[1]):
            return atomic_topics(edge[2])
        elif is_compound_propn(edge) or is_adjective_and_noun(edge):
            return atopics
        else:
            subtopics = set()
            for item in edge:
                subtopics |= atomic_topics(item)
            if len(subtopics) == 1:
                atopics = subtopics
            else:
                atopics |= subtopics
    return atopics

def atomic_topics_old(edge):
    atopics = atomic_topic(edge)
    if is_edge(edge):
        if is_compound_propn(edge) or is_adjective_and_noun(edge):
            return atopics
        for item in edge:
            atopics |= atomic_topics(item)
        # if len(atopics) > 0:
        #     return atopics
        # else:
        #     return atomic_topic(edge)
    return atopics

def atomic_topics2(edge):
    atopics = atomic_topic(edge)
    if is_edge(edge):
        if len(edge) == 3 and edge[0] == '+/gb' and is_adp(edge[1]):
            return atomic_topics2(edge[2])
            
        subtopics = set()
        for item in edge:
            subtopics |= atomic_topics2(item)
        if len(subtopics) == 1:
            atopics = subtopics
        else:
            atopics |= subtopics
    return atopics

### Entities, Actors and Conflicts

In [8]:
entities = {}
actors = {}
conflicts = {}
topic_actor = {}


def edge2actors(edge, check_actors=True):
    if is_edge(edge) and edge[0] == '+/gb':
        if is_adp(edge[1]) and len(edge) == 3:
            return edge2actors(edge[2])
        elif is_cconj(edge[1]) and len(edge) > 2:
            result = set()
            for item in edge[2:]:
                result |= edge2actors(item)
            return result
    if has_noun(edge):
        synact = syn.main_synonym(hg, edge, in_adp=True)
        if check_actors:
            if synact in actors and actors[synact] > 2:
                return {synact}
        else:
            return {synact}
    return set()


def add_to_table(table, item):
    if item not in table:
        table[item] = 0
    table[item] += 1


def add_entity(entity):
    if entity not in entities:
        entities[entity] = {'mentions': {},
                            'mentioned_by': {},
                            'conflict_towards': {},
                            'conflict_from': {},
                            'conflict_actors': []}

        
def add_mention(actor, concept):
    add_entity(actor)
    add_entity(concept)
    add_to_table(entities[actor]['mentions'], concept)
    add_to_table(entities[concept]['mentioned_by'], actor)
    

def add_conflict(orig, targ, concept):
    add_entity(orig)
    add_entity(targ)
    add_entity(concept)
    add_to_table(entities[orig]['conflict_towards'], targ)
    add_to_table(entities[targ]['conflict_from'], orig)
    entities[concept]['conflict_actors'].append((orig, targ))
   

def add_conflict_tuple(orig, targ, concept, source):
    if orig not in conflicts:
        conflicts[orig] = []
    if targ not in conflicts:
        conflicts[targ] = []
    conflict_tuple = (orig, targ, concept, source)
    conflicts[orig].append(conflict_tuple)
    conflicts[targ].append(conflict_tuple)

mention_edges = hg.pattern2edges(('mention/gb.inf', None, None))
conflict_edges = hg.pattern2edges(('conflict/gb.inf', None, None, None))
nclaims = len(mention_edges)
nconflicts = len(conflict_edges)
N = (nclaims + nconflicts) * 2

i = 0
with progressbar.ProgressBar(max_value=N) as bar:
    # find actors
    for edge in hg.pattern2edges(('mention/gb.inf', None, None)):
        origs = edge2actors(edge[1], check_actors=False)
        for orig in origs:
            if orig not in actors:
                actors[orig] = 0
            actors[orig] += 1
        i += 1
        bar.update(i)
        
    for edge in hg.pattern2edges(('conflict/gb.inf', None, None, None)):
        origs = edge2actors(edge[1], check_actors=False)
        for orig in origs:
            if orig not in actors:
                actors[orig] = 0
            actors[orig] += 1
        i += 1
        bar.update(i)


    # conflicts and mentions
    for edge in hg.pattern2edges(('mention/gb.inf', None, None)):
        origs = edge2actors(edge[1])
        topics = atomic_topics2(edge[2])
        for orig in origs:
            for topic in topics:
                add_mention(orig, topic)
                if topic not in topic_actor:
                    topic_actor[topic] = {}
                if orig not in topic_actor[topic]:
                    topic_actor[topic][orig] = {'related-topics': set(), 'edges': []}
                topic_actor[topic][orig]['related-topics'] |= topics - {topic}
                topic_actor[topic][orig]['edges'].append(edge[2])
        i += 1
        bar.update(i)
        
    for edge in hg.pattern2edges(('conflict/gb.inf', None, None, None)):
        origs = edge2actors(edge[1])
        targs = edge2actors(edge[2])
        topics = atomic_topics(edge[3])
        detailed_topic = choose_topic(edge[3])
        source = hg.pattern2edges(('source/gb.inf', edge, None)).pop()[2]
        for orig in origs:
            for targ in targs:
                add_conflict_tuple(orig, targ, detailed_topic, source)
                for topic in topics:
                    add_conflict(orig, targ, topic)
        i += 1
        bar.update(i)

print('claims: %s' % nclaims)
print('conflicts: %s' % nconflicts)

100% (31474 of 31474) |##################| Elapsed Time: 0:01:56 Time:  0:01:56


claims: 13100
conflicts: 2637


### Metrics

In [9]:
def compute_metrics(entity):
    mentioned_by = entities[entity]['mentioned_by']
    weights = [mentioned_by[item] for item in mentioned_by]
    total = sum(weights)
    h = 0.
    if total > 0:
        h_weights = [float(i) / float(total) for i in weights]
        h_weights = [i * i for i in h_weights]
        h = 1. / sum(h_weights)
    entities[entity]['h'] = h
    entities[entity]['total'] = total
    entities[entity]['degree'] = syn.synonyms_degree(hg, entity)
    
def metrics(concept):
    return {'h': entities[concept]['h'],
            'total': entities[concept]['total'],
            'degree': entities[concept]['degree']}

i = 0
with progressbar.ProgressBar(max_value=len(entities)) as bar:
    for entity in entities:
        compute_metrics(entity)
        i += 1
        bar.update(i)

100% (38612 of 38612) |##################| Elapsed Time: 0:00:36 Time:  0:00:36


### One Actor

In [10]:
# actor = syn_id('france')
actor = 'germany/syn12662'

lactor = hg.get_label(actor)
json_data = {}

for conflict_tuple in conflicts[actor]:
    other = conflict_tuple[0]
    direction = 'from'
    if other == actor:
        other = conflict_tuple[1]
        direction = 'to'
    topic = conflict_tuple[2]
    lother = hg.get_label(other)
    ltopic = syn_label(topic)
    if lother not in json_data:
        json_data[lother] = {}
    if ltopic not in json_data[lother]:
        json_data[lother][ltopic] = metrics(other)
        json_data[lother][ltopic]['from'] = 0
        json_data[lother][ltopic]['to'] = 0
    json_data[lother][ltopic][direction] += 1
    json_data[lother][ltopic]['source'] = conflict_tuple[3]

topics = {} 

for targ in json_data:
    concepts = set(json_data[targ].keys())
    for concept in concepts:
        if concept not in topics:
            topics[concept] = 0
        topics[concept] += 1
        
for targ in json_data:
    for concept in json_data[targ]:
        json_data[targ][concept]['local_degree'] = topics[concept]
        
with open('%s.json' % lactor, 'wt') as out:
    res = j.dump(json_data, out, sort_keys=False, indent=4, separators=(',', ': '))
    
json_data

{'turkeys': {'working against erdogan': {'h': 14.91167574107688,
   'total': 157,
   'degree': 3268,
   'from': 1,
   'to': 0,
   'source': ('accuses/nlp.accuse.verb',
    'turkey/nlp.turkey.propn',
    'germany/nlp.germany.propn',
    ('+/gb',
     'of/nlp.of.adp',
     ('working_against/nlp.work.verb+nlp.against.adp',
      'erdogan/nlp.erdogan.propn'))),
   'local_degree': 1},
  'backing failed coup': {'h': 14.91167574107688,
   'total': 157,
   'degree': 3268,
   'from': 1,
   'to': 0,
   'source': ('accuses/nlp.accuse.verb',
    'turkey/nlp.turkey.propn',
    'germany/nlp.germany.propn',
    ('+/gb',
     'of/nlp.of.adp',
     ('backing_failed/nlp.back.verb+nlp.fail.verb', 'coup/nlp.coup.noun'))),
   'local_degree': 1},
  'cultural racism over eu accession remarks': {'h': 14.91167574107688,
   'total': 157,
   'degree': 3268,
   'from': 1,
   'to': 0,
   'source': ('accuses/nlp.accuse.verb',
    'turkey/nlp.turkey.propn',
    'germany/nlp.germany.propn',
    ('+/gb',
     'of/nlp.

## Two Actors

In [17]:
def mentions_intersection(actors):
    intersection = None
    for actor in actors:
        concepts = set(entities[actor]['mentions'].keys())
        if intersection is None:
            intersection = concepts
        else:
            intersection &= concepts
    return intersection

def show_by_h(concepts, decreasing=True):
    concepts_h = {}
    for concept in concepts:
        concepts_h[concept] = entities[concept]['h']

    sorted_h = sorted(concepts_h.items(), key=operator.itemgetter(1), reverse=decreasing)

    for concept, h in sorted_h:
        print('%s [%s]' % (syn_label(concept), h))

def metrics_and_commons(concept, actor1, actor2):
    m = metrics(concept)
    atoms1 = topic_actor[concept][actor1]['related-topics']
    atoms2 = topic_actor[concept][actor2]['related-topics']
    common_related_atoms = tuple(atoms1 & atoms2)
    m['common_related_atoms'] = [syn_label(atom) for atom in common_related_atoms]
    m['common_related_atoms_count'] = len(m['common_related_atoms'])
    topics1 = topic_actor[concept][actor1]['edges']
    topics2 = topic_actor[concept][actor2]['edges']
    m['topics'] = [[choose_topic(syn_label(topic)) for topic in topics1],
                   [choose_topic(syn_label(topic)) for topic in topics2]]
    return m


actor1 = syn_id('russia')
actor2 = syn_id('u.s.')

actor1_name = hg.get_label(actor1)
actor2_name = hg.get_label(actor2)

inter = mentions_intersection([actor1, actor2])

data = {'actors': [actor1_name, actor2_name],
        'common_concepts': {}}

for concept in inter:
    data['common_concepts'][syn_label(concept)] = metrics_and_commons(concept, actor1, actor2)

file_name = '%s-%s.json' % (actor1_name, actor2_name)
with open(file_name, 'wt') as out:
    res = j.dump(data, out, sort_keys=False, indent=4, separators=(',', ': '))

data

{'actors': ['russia', 'u.s.'],
 'common_concepts': {'iran': {'h': 28.234856535600493,
   'total': 163,
   'degree': 4042,
   'common_related_atoms': ['syria', 'resolution'],
   'common_related_atoms_count': 2,
   'topics': [['nov. 24 deadline may not be met in iran talks interfax',
     'it is drafting roadmap to resolve syria conflict with turkey iran',
     'iran missile test does not contravene un resolution'],
    ["it 's not deterring foreign banks in iran",
     'recognition of israel not part of iran nuke deal',
     '$ 400 million to iran used as leverage for prisoners',
     'iran sent arms to iraq to fight isis',
     'iran complies with nuke deal but orders review on lifting sanctions',
     'it is considering response to iran ballistic missile test',
     "iran 's nuclear reactor plans deeply troubling",
     'iran has ceased controversial nuclear practice',
     'iran rocket test breaches u.n. resolution',
     'north korea can learn from iran nuclear deal',
     "iran wo 

### Conflict Topics

In [18]:
sorted_conflict_topics = sorted([(entity, entities[entity]['c']) for entity in entities], key=operator.itemgetter(1), reverse=True)

sorted_conflict_topics

KeyError: 'c'

### Conflict graph

In [13]:
topic = 'syria/syn11910'
conflict_pairs = entities[topic]['conflict_actors']
cactors = set()
for pair in conflict_pairs:
    cactors.add(syn_label(pair[0]))
    cactors.add(syn_label(pair[1]))

G = nx.DiGraph()
for actor in cactors:
    G.add_node(actor)
for edge in conflict_pairs:
    G.add_edge(syn_label(edge[0]), syn_label(edge[1]))
    
nx.write_gml(G, '%s.gml' % syn_label(topic))

### Factions

In [14]:
class Factions:
    def __init__(self, topic):
        self.topic = topic
        self.edges = {}
        self.degrees = {}
        self.main_conflict = None
        self.actors = set()
        self.factions = (set(), set(), set())
        
    def attacks_faction(self, actor, faction):
        for actor2 in self.factions[faction]:
            edge = tuple(sorted((actor, actor2)))
            if edge in self.edges:
                return True
        return False

    def assign_faction(self, actor):
        if actor in self.actors:
            return
        self.actors.add(actor)
        if self.attacks_faction(actor, 0) and not self.attacks_faction(actor, 1):
            self.factions[1].add(actor)
        elif self.attacks_faction(actor, 1) and not self.attacks_faction(actor, 0):
            self.factions[0].add(actor)
        else:
            self.factions[2].add(actor)

    def process(self):
        conflict_pairs = entities[self.topic]['conflict_actors']

        for pair in conflict_pairs:
            edge = tuple(sorted(pair))
            if edge not in self.edges:
                self.edges[edge] = 0
            actor1, actor2 = edge
            if actor1 not in self.degrees:
                self.degrees[actor1] = 0
            self.degrees[actor1] += 1
            if actor2 not in self.degrees:
                self.degrees[actor2] = 0
            self.degrees[actor2] += 1
    
        for edge in self.edges:
            self.edges[edge] = min([self.degrees[actor] for actor in edge])

        sorted_edges = sorted([(edge[0], edge[1], self.edges[edge]) for edge in self.edges], key=operator.itemgetter(2), reverse=True)

        self.main_conflict = sorted_edges[0]
        self.actors.add(self.main_conflict[0])
        self.actors.add(self.main_conflict[1])
        self.factions[0].add(self.main_conflict[0])
        self.factions[1].add(self.main_conflict[1])

        for edge in sorted_edges[1:]:
            self.assign_faction(edge[0])
            self.assign_faction(edge[1])
            
    def get_faction(self, actor):
        if actor in self.factions[0]:
            return -1
        elif actor in self.factions[1]:
            return 1
        else:
            return 0
        
f = Factions('syria/syn11910')
f.process()
print(f.main_conflict)

print('FACTION A')
for actor in f.factions[0]:
    print(syn_label(actor))
print('FACTION B')
for actor in f.factions[1]:
    print(syn_label(actor))
print('UNDECIDED')
for actor in f.factions[2]:
    print(syn_label(actor))

('russia/syn11969', 'u.s./syn26', 24)
FACTION A
iran
damascus
moscow
saudi
hezbollah
putin
china
russia
FACTION B
turkey pm
united states
obama
turkish pm
benjamin
france
germany
us
israel
syria opposition
UNDECIDED
assad
john
turkey
saudi king
un
ankara


### Generate topics data

In [15]:
json_data = {}

for topic, c in sorted_conflict_topics:
    if c > 1:
        f = Factions(topic)
        f.process()
    
        cactors = {}
        for actor in f.actors:
            cactor = {'faction': f.get_faction(actor), 'outdeg': 0, 'indeg': 0}
            lactor = syn_label(actor)
            cactors[lactor] = cactor
        
        for pair in entities[topic]['conflict_actors']:
            orig = syn_label(pair[0])
            targ = syn_label(pair[1])
            cactors[orig]['outdeg'] += 1
            cactors[targ]['indeg'] += 1
    
        main_conflict = [syn_label(f.main_conflict[0]), syn_label(f.main_conflict[1])]
    
        ltopic = syn_label(topic)
        json_data[ltopic] = {'conflictuality': c, 'actors': cactors, 'main_conflict': main_conflict}
        
with open('conflict_topics.json', 'wt') as out:
    res = j.dump(json_data, out, sort_keys=False, indent=4, separators=(',', ': '))

NameError: name 'sorted_conflict_topics' is not defined

In [48]:
edge = str2edge('(accuses/nlp.accuse.verb us/nlp.us.propn israel/nlp.israel.propn (+/gb of/nlp.of.adp (targeting/nlp.target.verb (+/gb of/nlp.of.adp family/nlp.family.noun (+/gb palestine/nlp.palestine.propn (murdered/nlp.murder.verb teen/nlp.teen.noun))))))')
hg.get_str_attribute(edge, 'text')
edge2str(without_namespaces(edge))

'(accuses us israel (+ of (targeting (+ of family (+ palestine (murdered teen))))))'

## Validation

In [21]:
all_claims = []
all_conflicts = []

for edge in hg.pattern2edges(('mention/gb.inf', None, None)):
    origs = edge2actors(edge[1])
    topics = atomic_topics2(edge[2])
    source = hg.pattern2edges(('source/gb.inf', edge, None)).pop()[2]
    text = hg.get_str_attribute(source, 'text')
    all_claims.append({'origs': origs,
                       'topics': topics,
                       'orig_raw': edge[1],
                       'topic_raw': edge[2],
                       'detailed_topic': choose_topic(edge[2]),
                       'sentence': text,
                       'edge': source})
    
    
for edge in hg.pattern2edges(('conflict/gb.inf', None, None, None)):
    origs = edge2actors(edge[1])
    targs = edge2actors(edge[2])
    topics = atomic_topics(edge[3])
    source = hg.pattern2edges(('source/gb.inf', edge, None)).pop()[2]
    text = hg.get_str_attribute(source, 'text')
    all_conflicts.append({'origs': origs,
                          'targs': targs,
                          'topics': topics,
                          'orig_raw': edge[1],
                          'targ_raw': edge[2],
                          'topic_raw': edge[3],
                          'detailed_topic': choose_topic(edge[3]),
                          'sentence': text,
                          'edge': source})

In [29]:
sample_claims = []
while len(sample_claims) < 100:
    choice = random.choice(all_claims)
    if choice not in sample_claims and choice['origs']:
        sample_claims.append(choice)

sample_conflicts = []
while len(sample_conflicts) < 100:
    choice = random.choice(all_conflicts)
    if choice not in sample_claims and choice['origs'] and choice['targs']:
        sample_conflicts.append(choice)

In [23]:
sample_claims[0]

{'origs': {'defense_chief/syn4184'},
 'topics': {'asia/syn12203',
  "gov't/syn512",
  'to_stay_put_in/syn2167251',
  'to_stay_put_in_asia/syn2167250',
  'trump/syn202',
  "trump_gov't/syn2167249",
  "trump_gov't_to_stay_put_in_asia/syn2167248"},
 'orig_raw': ('+/gb', 'defense/nlp.defense.noun', 'chief/nlp.chief.noun'),
 'topic_raw': ('+/gb',
  ('+/gb', 'trump/nlp.trump.propn', "gov't/nlp.gov't.verb"),
  ('to_stay_put_in/nlp.to.part+nlp.stay.verb+nlp.put.verb+nlp.in.adp',
   'asia/nlp.asia.propn')),
 'detailed_topic': "trump_gov't_to_stay_put_in_asia/syn2167248",
 'sentence': "Trump gov't to stay put in Asia, defense chief says.",
 'edge': ('says/nlp.say.verb',
  ('+/gb', 'defense/nlp.defense.noun', 'chief/nlp.chief.noun'),
  ('+/gb',
   ('+/gb', 'trump/nlp.trump.propn', "gov't/nlp.gov't.verb"),
   ('to_stay_put_in/nlp.to.part+nlp.stay.verb+nlp.put.verb+nlp.in.adp',
    'asia/nlp.asia.propn')))}

In [30]:
with open('claim_validation.csv', 'w') as f:
    f.write('sentence;edge;orig_raw;topic_raw;origs;topics;detailed_topic\n')
    for item in sample_claims:
        row = []
        row.append(item['sentence'].replace(';', ','))
        row.append(edge2str(item['edge'], namespaces=False))
        row.append(edge2str(item['orig_raw'], namespaces=False))
        row.append(edge2str(item['topic_raw'], namespaces=False))
        row.append('|'.join([syn_label(x) for x in item['origs']]))
        row.append('|'.join([syn_label(x) for x in item['topics']]))
        row.append(syn_label(item['detailed_topic']))
        f.write('%s\n' % ';'.join(row))

In [28]:
with open('conflict_validation.csv', 'w') as f:
    f.write('sentence;edge;orig_raw;targ_raw;topic_raw;origs;targs;topics;detailed_topic\n')
    for item in sample_conflicts:
        row = []
        row.append(item['sentence'].replace(';', ','))
        row.append(edge2str(item['edge'], namespaces=False))
        row.append(edge2str(item['orig_raw'], namespaces=False))
        row.append(edge2str(item['targ_raw'], namespaces=False))
        row.append(edge2str(item['topic_raw'], namespaces=False))
        row.append('|'.join([syn_label(x) for x in item['origs']]))
        row.append('|'.join([syn_label(x) for x in item['targs']]))
        row.append('|'.join([syn_label(x) for x in item['topics']]))
        row.append(syn_label(item['detailed_topic']))
        f.write('%s\n' % ';'.join(row))